# Join the tables

### Import libraries

In [1]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import os
import getpass
import csv

### Read from local

In [3]:
sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()
conf = SparkConf().set("spark.driver.memory", "8g")

In [4]:
# read from local
rent_df = ss.read.csv('data/rent_df.csv', header=True)
census_2015_df = ss.read.csv('data/census_2015_df.csv', header=True)
county_2015_df = ss.read.csv('data/county_2015_df.csv', header=True)
census_2017_df = ss.read.csv('data/census_2017_df.csv', header=True)
county_2017_df = ss.read.csv('data/county_2017_df.csv', header=True)
benefits_cost_sharing_df = ss.read.csv('data/benefits_cost_sharing_df.csv', header=True)
business_rules_df = ss.read.csv('data/business_rules_df.csv', header=True)
network_df = ss.read.csv('data/network_df.csv', header=True)
plan_attributes_df = ss.read.csv('data/plan_attributes_df.csv', header=True)
rate_df = ss.read.csv('data/rate_df.csv', header=True)
service_area_df = ss.read.csv('data/service_area_df.csv', header=True)

In [5]:
# rent_df + county_2015_df + benefits_cost_sharing_df
# year 2015 inner join on State / stateCode / RegionName

In [6]:
%%time

# filtering tables 
rent_df = rent_df.filter(rent_df.Year == 2015)
benefits_cost_sharing_df = benefits_cost_sharing_df.filter(benefits_cost_sharing_df.BusinessYear == 2015)
business_rules_df = business_rules_df.filter(business_rules_df.BusinessYear == 2015)
network_df = network_df.filter(network_df.BusinessYear == 2015)
plan_attributes_df = plan_attributes_df.filter(plan_attributes_df.BusinessYear == 2015)
rate_df = rate_df.filter(rate_df.BusinessYear == 2015)
service_area_df = service_area_df.filter(service_area_df.BusinessYear == 2015)

CPU times: user 5.65 ms, sys: 1.92 ms, total: 7.57 ms
Wall time: 94.4 ms


In [7]:
%%time

# joining tables
# df = rent_df.join(county_2015_df, rent_df.RegionName == county_2015_df.State, 'left_outer').drop(county_2015_df.County)
# df = df.join(benefits_cost_sharing_df, ['StateCode'], 'left_outer')
# df = df.join(business_rules_df, ['StateCode', 'BusinessYear', 'ImportDate', 'IssuerId', 'IssuerId2', 'RowNumber', 'SourceName', 'StandardComponentId', 'VersionNum'], 'left_outer')
# df = df.join(network_df, ['StateCode', 'BusinessYear', 'DentalOnlyPlan', 'ImportDate', 'IssuerId', 'IssuerId2', 'MarketCoverage', 'RowNumber', 'SourceName', 'StateCode2', 'VersionNum'], 'left_outer')
# df = df.join(plan_attributes_df, ['StateCode', 'BusinessYear', 'DentalOnlyPlan', 'ImportDate', 'IssuerId', 'IssuerId2', 'MarketCoverage', 'NetworkId', 'PlanId', 'RowNumber', 'SourceName', 'StandardComponentId', 'StateCode2', 'TIN', 'VersionNum'], 'left_outer')
# df = df.join(rate_df, ['StateCode', 'BusinessYear', 'ImportDate', 'IssuerId', 'IssuerId2', 'PlanId', 'RowNumber', 'SourceName', 'VersionNum'], 'left_outer')
# df = df.join(service_area_df, ['StateCode', 'BusinessYear', 'DentalOnlyPlan', 'ImportDate', 'IssuerId', 'IssuerId2', 'MarketCoverage', 'RowNumber', 'ServiceAreaId', 'SourceName', 'StateCode2', 'VersionNum'], 'left_outer')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [8]:
%%time

# joining tables
df = rent_df.join(county_2015_df, rent_df.RegionName == county_2015_df.State, 'inner').drop(county_2015_df.County)
df = df.join(benefits_cost_sharing_df, ['StateCode'], 'inner')
df = df.join(business_rules_df, ['StateCode'], 'inner')
df = df.join(network_df, ['StateCode'], 'inner')
# df = df.join(plan_attributes_df, ['StateCode', 'BusinessYear', 'DentalOnlyPlan', 'ImportDate', 'IssuerId', 'IssuerId2', 'MarketCoverage', 'NetworkId', 'PlanId', 'RowNumber', 'SourceName', 'StandardComponentId', 'StateCode2', 'TIN', 'VersionNum'], 'left_outer')
# df = df.join(rate_df, ['StateCode', 'BusinessYear', 'ImportDate', 'IssuerId', 'IssuerId2', 'PlanId', 'RowNumber', 'SourceName', 'VersionNum'], 'left_outer')
# df = df.join(service_area_df, ['StateCode', 'BusinessYear', 'DentalOnlyPlan', 'ImportDate', 'IssuerId', 'IssuerId2', 'MarketCoverage', 'RowNumber', 'ServiceAreaId', 'SourceName', 'StateCode2', 'VersionNum'], 'left_outer')

CPU times: user 3.32 ms, sys: 1.4 ms, total: 4.71 ms
Wall time: 57.1 ms


In [9]:
# sorted(df.columns)

In [10]:
%%time
df.count()

Py4JJavaError: An error occurred while calling o124.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 97 in stage 16.0 failed 1 times, most recent failure: Lost task 97.0 in stage 16.0 (TID 202, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 2188370 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1876)
	at org.apache.spark.sched

In [10]:
%%time
df.show(1)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/andrewyoung/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/andrewyoung/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/Users/andrewyoung/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o117.showString